In [134]:
import os, sys
MYPROJECT = '/Users/ryuki/Projects/graduation_research'
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")

import django
django.setup()

#### do below script when your mysql has gone away

In [143]:
from django import db
db.close_old_connections()

In [63]:
import pandas as pd

In [18]:
import MeCab
# import mojimoji
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

text = '解析したいテキスト'
mecab.parse('')#文字列がGCされるのを防ぐ

'EOS\n'

In [120]:
mecab.parse('湘南ベルマーレ世田谷線').split('\t')

['湘南ベルマーレ',
 '名詞,固有名詞,一般,*,*,*,湘南ベルマーレ,ショウナンベルマーレ,ショーナンベルマーレ\n世田谷線',
 '名詞,固有名詞,一般,*,*,*,世田谷線,セタガヤセン,セタガヤセン\nEOS\n']

参考 https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja


In [29]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize c
hoonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [137]:
from review_based_recommender.models import Spot, Review, City
import django_pandas

In [21]:
from django_pandas.io import read_frame
reviews = Review.objects.all()
review_df = read_frame(reviews, fieldnames=['id', 'username', 'title', 'content', 'spot_id'], verbose=False)

In [22]:
review_df

,id,username,title,content,spot_id
0,1,niramechan,ボードウォーク閉鎖中,観音崎京急ホテルやその施設を利用するなら京急の馬堀海岸から送迎バスが出ています。ボードウォー...,23
1,2,yukof186,ハイキングでした…,美術館から足をのばして行きましたが、思ったより急な登りが沢山。軽い散歩のつもりでいったら、わ...,23
2,3,隆幸 末,楽しかった！,車は観音崎第一駐車場に止めました。海のすぐそばです。平日は無料のようです。駐車場から遊歩道を...,23
3,4,さすらいのとんしゅけ,海好きにはたまりません,ここのオススメは浦賀水道を通るお船を眺める事。一日にものすごい数の船が通るのです。タンカーや...,23
4,5,ya1201,海岸散歩,京急ホテル宿泊の時はいつも朝海岸散歩します。\n東京湾入口行き交う船を間近に見ながらの散歩は...,23
5,6,bjnoche,爽やかな潮風、自然での戯れ、モダンな施設、おいしい食事、、、癒しの場,観音崎は都心から最も近い自然浜のあるビーチリゾートとして人気があり、また地元の人にも海水浴や...,23
6,7,とよとよ25,なかなか楽しめます,横須賀の南にある岬で、中に入れる灯台や戦跡が残されており、観光地としてお勧めです。このあたり...,23
7,8,yoitoko137,東京湾に突き出した岬,三浦半島にある東京湾に突き出した岬で、東京湾を行き来する船舶がよく見える場所です。歴史のある...,23
8,9,Adam756,夏場はBBQ客がいっぱい,横須賀中央駅から京急バスで30分ぐらいで着きました。海水浴、BBQ、灯台を見に行ったり自然遊...,23
9,10,ずんだれ,横須賀を代表する景勝地,横須賀市歌は「白波は白波は岬にくだけ、光る風光る風大地に遊ぶ」という歌詞で始まりますが、まさ...,23


In [60]:
def get_reviews_list(df):
    res = df.loc[:]['content'].tolist()
    return res
df = review_df.groupby('spot_id').apply(get_reviews_list).to_frame('reviews').reset_index()

In [61]:
df

,spot_id,reviews
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...


In [64]:
def reviews2tokens(reviews, stopwords):
    tokens = []
    for review in reviews:
        print(review)
        node = mecab.parseToNode(normalize_neologd(review))
        while node:
            word = node.surface
            if word in stopwords:
                continue
            print(node.feature)
            pos = node.feature.split(",")[1]
            tokens.append(word)
            node = node.next
    return tokens

In [65]:
def reviews2tokensbysimpleparse(reviews, stopwords):
    tokens = []
    for review in reviews:
        for chunk in mecab.parse(review).splitlines()[:-1]:
            (surface, feature) = chunk.split('\t')
            if feature.startswith('名詞'):
                print(feature)
                tokens.append(surface)
    return tokens

get stop word

In [66]:
import urllib.request
slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file  = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file if line.decode("utf-8").strip() != '']
slothlib_stopwords

['あそこ',
 'あたり',
 'あちら',
 'あっち',
 'あと',
 'あな',
 'あなた',
 'あれ',
 'いくつ',
 'いつ',
 'いま',
 'いや',
 'いろいろ',
 'うち',
 'おおまか',
 'おまえ',
 'おれ',
 'がい',
 'かく',
 'かたち',
 'かやの',
 'から',
 'がら',
 'きた',
 'くせ',
 'ここ',
 'こっち',
 'こと',
 'ごと',
 'こちら',
 'ごっちゃ',
 'これ',
 'これら',
 'ごろ',
 'さまざま',
 'さらい',
 'さん',
 'しかた',
 'しよう',
 'すか',
 'ずつ',
 'すね',
 'すべて',
 'ぜんぶ',
 'そう',
 'そこ',
 'そちら',
 'そっち',
 'そで',
 'それ',
 'それぞれ',
 'それなり',
 'たくさん',
 'たち',
 'たび',
 'ため',
 'だめ',
 'ちゃ',
 'ちゃん',
 'てん',
 'とおり',
 'とき',
 'どこ',
 'どこか',
 'ところ',
 'どちら',
 'どっか',
 'どっち',
 'どれ',
 'なか',
 'なかば',
 'なに',
 'など',
 'なん',
 'はじめ',
 'はず',
 'はるか',
 'ひと',
 'ひとつ',
 'ふく',
 'ぶり',
 'べつ',
 'へん',
 'ぺん',
 'ほう',
 'ほか',
 'まさ',
 'まし',
 'まとも',
 'まま',
 'みたい',
 'みつ',
 'みなさん',
 'みんな',
 'もと',
 'もの',
 'もん',
 'やつ',
 'よう',
 'よそ',
 'わけ',
 'わたし',
 'ハイ',
 '上',
 '中',
 '下',
 '字',
 '年',
 '月',
 '日',
 '時',
 '分',
 '秒',
 '週',
 '火',
 '水',
 '木',
 '金',
 '土',
 '国',
 '都',
 '道',
 '府',
 '県',
 '市',
 '区',
 '町',
 '村',
 '各',
 '第',
 '方',
 '何',
 '的',
 '度',
 '文',
 '者',
 '性',
 '体',
 '人',
 '他',
 '今',
 

↓時間かかるよ
参考： https://qiita.com/yukinoi/items/81a707c1317c97f5fdf9

In [67]:
stopwords = ['\n']
stopwords += slothlib_stopwords
df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokens(doc, stopwords))

最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありますが、駅間の移動は楽です。カフェなんかがもう少し多いとベターかな。
BOS/EOS,*,*,*,*,*,*,*,*
名詞,副詞可能,*,*,*,*,最近,サイキン,サイキン
記号,読点,*,*,*,*,、,、,、
連体詞,*,*,*,*,*,この,コノ,コノ


KeyboardInterrupt: 

**parse** faster than **parsetonode**

In [104]:
%%time
stopwords = ['\n','、','「','」']
stopwords += slothlib_stopwords
df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokensbysimpleparse(doc, stopwords))

名詞,副詞可能,*,*,*,*,最近,サイキン,サイキン
名詞,一般,*,*,*,*,あたり,アタリ,アタリ
名詞,サ変接続,*,*,*,*,仕事,シゴト,シゴト
名詞,一般,*,*,*,*,プライベート,プライベート,プライベート
名詞,一般,*,*,*,*,用事,ヨウジ,ヨージ
名詞,一般,*,*,*,*,エリア,エリア,エリア
名詞,一般,*,*,*,*,ビル,ビル,ビル
名詞,接尾,一般,*,*,*,だらけ,ダラケ,ダラケ
名詞,一般,*,*,*,*,駅,エキ,エキ
名詞,接尾,一般,*,*,*,間,カン,カン
名詞,サ変接続,*,*,*,*,移動,イドウ,イドー
名詞,形容動詞語幹,*,*,*,*,楽,ラク,ラク
名詞,一般,*,*,*,*,カフェ,カフェ,カフェ
名詞,一般,*,*,*,*,ベター,ベター,ベター
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,一般,*,*,*,*,周辺,シュウヘン,シューヘン
名詞,非自立,副詞可能,*,*,*,ところ,トコロ,トコロ
名詞,一般,*,*,*,*,一手,イッテ,イッテ
名詞,一般,*,*,*,*,水辺,ミズベ,ミズベ
名詞,一般,*,*,*,*,雰囲気,フンイキ,フンイキ
名詞,一般,*,*,*,*,最高,サイコウ,サイコー
名詞,副詞可能,*,*,*,*,平日,ヘイジツ,ヘイジツ
名詞,副詞可能,*,*,*,*,午後,ゴゴ,ゴゴ
名詞,一般,*,*,*,*,ドリンク,ドリンク,ドリンク
名詞,サ変接続,*,*,*,*,軽食,ケイショク,ケイショク
名詞,一般,*,*,*,*,公園,コウエン,コーエン
名詞,非自立,一般,*,*,*,の,ノ,ノ
名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
名詞,一般,*,*,*,*,一帯,イッタイ,イッタイ
名詞,副詞可能,*,*,*,*,昔,ムカシ,ムカシ
名詞,固有名詞,一般,*,*,*,造船所,ゾウセンジョ,ゾーセンジョ
名詞,副詞可能,*,*,*,*,今,イマ,イマ
名詞,非自立,助動詞語幹,*,*,*,様,ヨウ,ヨー
名詞,一般,*,*,*,*,風情,フゼイ,フゼイ
名詞,一般,*,*,*,*,面,メン,メン
名詞,一般,*,*,*,*,今一つ,イマヒトツ,イマヒトツ
名詞,一般,*,*,*,*,空き地,

名詞,一般,*,*,*,*,ホテル,ホテル,ホテル
名詞,固有名詞,地域,一般,*,*,遊園地,ユウエンチ,ユーエンチ
名詞,一般,*,*,*,*,マンション,マンション,マンション
名詞,サ変接続,*,*,*,*,開発,カイハツ,カイハツ
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,一般,*,*,*,*,*
名詞,一般,*,*,*,*,映画,エイガ,エイガ
名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
名詞,副詞可能,*,*,*,*,途中,トチュウ,トチュー
名詞,一般,*,*,*,*,あたり,アタリ,アタリ
名詞,一般,*,*,*,*,ジョギング,ジョギング,ジョギング
名詞,一般,*,*,*,*,人,ヒト,ヒト
名詞,一般,*,*,*,*,犬,イヌ,イヌ
名詞,サ変接続,*,*,*,*,散歩,サンポ,サンポ
名詞,一般,*,*,*,*,人,ヒト,ヒト
名詞,サ変接続,*,*,*,*,共通,キョウツウ,キョーツー
名詞,代名詞,一般,*,*,*,みんな,ミンナ,ミンナ
名詞,接尾,一般,*,*,*,街,ガイ,ガイ
名詞,サ変接続,*,*,*,*,リラックス,リラックス,リラックス
名詞,サ変接続,*,*,*,*,リラックス,リラックス,リラックス
名詞,非自立,助動詞語幹,*,*,*,よう,ヨウ,ヨー
名詞,一般,*,*,*,*,都内,トナイ,トナイ
名詞,非自立,助動詞語幹,*,*,*,よう,ヨウ,ヨー
名詞,固有名詞,一般,*,*,*,1日,ツイタチ,ツイタチ
名詞,接尾,副詞可能,*,*,*,中,チュウ,チュー
名詞,一般,*,*,*,*,パイプオルガン,パイプオルガン,パイプオルガン
名詞,一般,*,*,*,*,ホール,ホール,ホール
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,一般,*,*,*,*,ホール,ホール,ホール
名詞,一般,*,*,*,*,ホール,ホール,ホール
名詞,一般,*,*,*,*,音響,オンキョウ,オンキョー
名詞,一般,*,*,*,*,効果,コウカ,コーカ
名詞,一般,*,*,*,*,パイプオルガン,パイプオルガン,パイプオルガン
名詞,副詞可能,*,*,*,*,前,マエ,マエ
名詞,一般,*,*,*,*,席,セキ,セキ
名詞,一般,*,*

名詞,一般,*,*,*,*,地方,チホウ,チホー
名詞,一般,*,*,*,*,海外,カイガイ,カイガイ
名詞,一般,*,*,*,*,友達,トモダチ,トモダチ
名詞,サ変接続,*,*,*,*,案内,アンナイ,アンナイ
名詞,一般,*,*,*,*,場所,バショ,バショ
名詞,副詞可能,*,*,*,*,朝,アサ,アサ
名詞,一般,*,*,*,*,昼夜,チュウヤ,チューヤ
名詞,一般,*,*,*,*,雨,アメ,アメ
名詞,一般,*,*,*,*,表情,ヒョウジョウ,ヒョージョー
名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
名詞,サ変接続,*,*,*,*,デート,デート,デート
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,形容動詞語幹,*,*,*,*,綺麗,キレイ,キレイ
名詞,一般,*,*,*,*,街並み,マチナミ,マチナミ
名詞,一般,*,*,*,*,海,ウミ,ウミ
名詞,形容動詞語幹,*,*,*,*,綺麗,キレイ,キレイ
名詞,一般,*,*,*,*,夕焼け,ユウヤケ,ユーヤケ
名詞,サ変接続,*,*,*,*,ショッピング,ショッピング,ショッピング
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,ナイ形容詞語幹,*,*,*,*,問題,モンダイ,モンダイ
名詞,一般,*,*,*,*,街,マチ,マチ
名詞,一般,*,*,*,*,街,マチ,マチ
名詞,非自立,一般,*,*,*,の,ノ,ノ
名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
名詞,接尾,形容動詞語幹,*,*,*,的,テキ,テキ
名詞,非自立,副詞可能,*,*,*,ため,タメ,タメ
名詞,固有名詞,一般,*,*,*,オフィス街,オフィスガイ,オフィスガイ
名詞,一般,*,*,*,*,遊び,アソビ,アソビ
名詞,一般,*,*,*,*,場所,バショ,バショ
名詞,固有名詞,一般,*,*,*,観光地,カンコウチ,カンコーチ
名詞,一般,*,*,*,*,テレビ,テレビ,テレビ
名詞,固有名詞,一般,*,*,*,観光地,カンコウチ,カンコーチ
名詞,サ変接続,*,*,*,*,紹介,ショウカイ,ショーカイ
名詞,固有名詞,一般,*,*,*,富士山,フジサン,フジサン
名詞,一般,*,*,*,*,絶景,ゼッケイ,ゼッケイ
名詞,一般,*,*,

名詞,固有名詞,一般,*,*,*,ジモティ,ジモティ,ジモティ
名詞,非自立,一般,*,*,*,もの,モノ,モノ
名詞,代名詞,一般,*,*,*,ここ,ココ,ココ
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,一般,*,*,*,*,生まれ,ウマレ,ウマレ
名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
名詞,固有名詞,一般,*,*,*,何度,ナンド,ナンド
名詞,固有名詞,一般,*,*,*,食事処,ショクジドコロ,ショクジドコロ
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,一般,*,*,*,*,人,ヒト,ヒト
名詞,非自立,一般,*,*,*,ん,ン,ン
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,非自立,副詞可能,*,*,*,とき,トキ,トキ
名詞,代名詞,一般,*,*,*,ここ,ココ,ココ
名詞,固有名詞,一般,*,*,*,中華街,チュウカガイ,チューカガイ
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,非自立,一般,*,*,*,事,コト,コト
名詞,形容動詞語幹,*,*,*,*,重要,ジュウヨウ,ジューヨー
名詞,固有名詞,一般,*,*,*,観光スポット,カンコウスポット,カンコースポット
名詞,固有名詞,一般,*,*,*,ランドマークタワー,ランドマークタワー,ランドマークタワー
名詞,固有名詞,一般,*,*,*,日本丸,ニッポンマル,ニッポンマル
名詞,固有名詞,地域,一般,*,*,汽車道,キシャミチ,キシャミチ
名詞,固有名詞,一般,*,*,*,ワールドポーターズ,ワールドポーターズ,ワールドポーターズ
名詞,固有名詞,一般,*,*,*,カップヌードルミュージアム,カップヌードルミュージアム,カップヌードルミュージアム
名詞,固有名詞,一般,*,*,*,アンパンマンミュージアム,アンパンマンミュージアム,アンパンマンミュージアム
名詞,固有名詞,一般,*,*,*,コスモワールド,コスモワールド,コスモワールド
名詞,固有名詞,地域,一般,*,*,大観覧車,ダイカンランシャ,ダイカンランシャ
名詞,固有名詞,一般,*,*,*,赤レンガ倉庫,アカレンガソウコ,アカレンガソーコ
名詞,固有名詞,一般,*,*,*,山下公園,ヤマシタコウエン,ヤマシタコーエン
名詞,形容

名詞,一般,*,*,*,*,景色,ケシキ,ケシキ
名詞,形容動詞語幹,*,*,*,*,綺麗,キレイ,キレイ
名詞,固有名詞,一般,*,*,*,デートスポット,デートスポット,デートスポット
名詞,一般,*,*,*,*,スポット,スポット,スポット
名詞,サ変接続,*,*,*,*,点在,テンザイ,テンザイ
名詞,サ変接続,*,*,*,*,散歩,サンポ,サンポ
名詞,固有名詞,一般,*,*,*,桜木町,サクラギチョウ,サクラギチョー
名詞,固有名詞,一般,*,*,*,中華街,チュウカガイ,チューカガイ
名詞,一般,*,*,*,*,夜景,ヤケイ,ヤケイ
名詞,一般,*,*,*,*,最高,サイコウ,サイコー
名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,一般,*,*,*,*,地区,チク,チク
名詞,一般,*,*,*,*,クリスマス,クリスマス,クリスマス
名詞,副詞可能,*,*,*,*,時期,ジキ,ジキ
名詞,一般,*,*,*,*,クリスマス,クリスマス,クリスマス
名詞,一般,*,*,*,*,イルミネーション,イルミネーション,イルミネーション
名詞,一般,*,*,*,*,一色,イッショク,イッショク
名詞,形容動詞語幹,*,*,*,*,ロマンチック,ロマンチック,ロマンチック
名詞,固有名詞,一般,*,*,*,観覧車,カンランシャ,カンランシャ
名詞,一般,*,*,*,*,ライトアップ,ライトアップ,ライトアップ
名詞,副詞可能,*,*,*,*,時期,ジキ,ジキ
名詞,固有名詞,一般,*,*,*,時間帯,ジカンタイ,ジカンタイ
名詞,一般,*,*,*,*,見どころ,ミドコロ,ミドコロ
名詞,サ変接続,*,*,*,*,満載,マンサイ,マンサイ
名詞,一般,*,*,*,*,カップル,カップル,カップル
名詞,一般,*,*,*,*,家族,カゾク,カゾク
名詞,数,*,*,*,*,１,イチ,イチ
名詞,接尾,助数詞,*,*,*,人,ニン,ニン
名詞,一般,*,*,*,*,場所,バショ,バショ
名詞,一般,*,*,*,*,都内,トナイ,トナイ
名詞,一般,*,*,*,*,電車,デンシャ,デンシャ
名詞,固有名詞,一般,*,*,*,1時間,イチジカン,イチジカン
名詞,接尾,一般,

...

名詞,サ変接続,*,*,*,*,点在,テンザイ,テンザイ
名詞,サ変接続,*,*,*,*,散策,サンサク,サンサク
名詞,一般,*,*,*,*,エリア,エリア,エリア
名詞,一般,*,*,*,*,赤レンガ,アカレンガ,アカレンガ
名詞,サ変接続,*,*,*,*,買物,カイモノ,カイモノ
名詞,サ変接続,*,*,*,*,飲食,インショク,インショク
名詞,固有名詞,地域,一般,*,*,遊園地,ユウエンチ,ユーエンチ
名詞,一般,*,*,*,*,温泉,オンセン,オンセン
名詞,固有名詞,組織,*,*,*,馬車道,バシャミチ,バシャミチ
名詞,サ変接続,*,*,*,*,散歩,サンポ,サンポ
名詞,一般,*,*,*,*,映画館,エイガカン,エイガカン
名詞,一般,*,*,*,*,ワールド,ワールド,ワールド
名詞,一般,*,*,*,*,ポーター,ポーター,ポーター
名詞,サ変接続,*,*,*,*,買物,カイモノ,カイモノ
名詞,固有名詞,一般,*,*,*,海浜公園,カイヒンコウエン,カイヒンコーエン
名詞,非自立,一般,*,*,*,事,コト,コト
名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
名詞,一般,*,*,*,*,場所,バショ,バショ
名詞,一般,*,*,*,*,景色,ケシキ,ケシキ
名詞,サ変接続,*,*,*,*,デート,デート,デート
名詞,一般,*,*,*,*,尾,オ,オ
名詞,一般,*,*,*,*,目玉,メダマ,メダマ
名詞,固有名詞,一般,*,*,*,商業施設,ショウギョウシセツ,ショーギョーシセツ
名詞,一般,*,*,*,*,ベビーカー,ベビーカー,ベビーカー
名詞,非自立,副詞可能,*,*,*,ところ,トコロ,トコロ
名詞,副詞可能,*,*,*,*,平日,ヘイジツ,ヘイジツ
名詞,副詞可能,*,*,*,*,週末,シュウマツ,シューマツ
名詞,固有名詞,一般,*,*,*,全般的,ゼンパンテキ,ゼンパンテキ
名詞,サ変接続,*,*,*,*,施設,シセツ,シセツ
名詞,形容動詞語幹,*,*,*,*,きれい,キレイ,キレイ
名詞,一般,*,*,*,*,子連れ,コヅレ,コズレ
名詞,非自立,副詞可能,*,*,*,ところ,トコロ,トコロ
名詞,非自立,一般,*,*,*,方,ホウ,ホー
名詞,サ変接続,*,*,*,*,食事,ショクジ,ショクジ
名詞,

名詞,接尾,副詞可能,*,*,*,ごろ,ゴロ,ゴロ
名詞,サ変接続,*,*,*,*,点灯,テントウ,テントー
名詞,数,*,*,*,*,４,ヨン,ヨン
名詞,接尾,助数詞,*,*,*,時,ジ,ジ
名詞,数,*,*,*,*,５,ゴ,ゴ
名詞,数,*,*,*,*,０,ゼロ,ゼロ
名詞,接尾,助数詞,*,*,*,分,フン,フン
名詞,数,*,*,*,*,５,ゴ,ゴ
名詞,接尾,助数詞,*,*,*,時,ジ,ジ
名詞,一般,*,*,*,*,日没,ニチボツ,ニチボツ
名詞,副詞可能,*,*,*,*,直後,チョクゴ,チョクゴ
名詞,副詞可能,*,*,*,*,今回,コンカイ,コンカイ
名詞,数,*,*,*,*,３,サン,サン
名詞,接尾,助数詞,*,*,*,脚,キャク,キャク
名詞,非自立,一般,*,*,*,の,ノ,ノ
名詞,一般,*,*,*,*,写真,シャシン,シャシン
名詞,一般,*,*,*,*,場所,バショ,バショ
名詞,一般,*,*,*,*,夜景,ヤケイ,ヤケイ
名詞,固有名詞,一般,*,*,*,桜木町,サクラギチョウ,サクラギチョー
名詞,一般,*,*,*,*,駅,エキ,エキ
名詞,副詞可能,*,*,*,*,夕暮れ,ユウグレ,ユーグレ
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,形容動詞語幹,*,*,*,*,きれい,キレイ,キレイ
名詞,副詞可能,*,*,*,*,今回,コンカイ,コンカイ
名詞,固有名詞,一般,*,*,*,動く歩道,ウゴクホドウ,ウゴクホドー
名詞,固有名詞,地域,一般,*,*,ランドマーク,ランドマーク,ランドマーク
名詞,一般,*,*,*,*,ライトアップ,ライトアップ,ライトアップ
名詞,副詞可能,*,*,*,*,一昨年,イッサクネン,イッサクネン
名詞,副詞可能,*,*,*,*,年末,ネンマツ,ネンマツ
名詞,一般,*,*,*,*,ホテル,ホテル,ホテル
名詞,一般,*,*,*,*,カウントダウン,カウントダウン,カウントダウン
名詞,固有名詞,一般,*,*,*,一年,イチネン,イチネン
名詞,接尾,副詞可能,*,*,*,中,チュウ,チュー
名詞,非自立,副詞可能,*,*,*,ところ,トコロ,トコロ
名詞,一般,*,*,*,*,クリスマス,クリスマス,クリスマス
名詞,一般,*,*,*,*,イルミネーション,

名詞,一般,*,*,*,*,範囲,ハンイ,ハンイ
名詞,一般,*,*,*,*,穴場,アナバ,アナバ
名詞,固有名詞,一般,*,*,*,神奈川県警察,カナガワケンケイサツ,カナガワケンケイサツ
名詞,一般,*,*,*,*,本部,ホンブ,ホンブ
名詞,固有名詞,一般,*,*,*,電話予約,デンワヨヤク,デンワヨヤク
名詞,形容動詞語幹,*,*,*,*,必要,ヒツヨウ,ヒツヨー
名詞,サ変接続,*,*,*,*,展望,テンボウ,テンボー
名詞,一般,*,*,*,*,ロビー,ロビー,ロビー
名詞,固有名詞,一般,*,*,*,ランドマーク,ランドマーク,ランドマーク
名詞,サ変接続,*,*,*,*,眺望,チョウボウ,チョーボー
名詞,固有名詞,一般,*,*,*,日本丸,ニッポンマル,ニッポンマル
名詞,固有名詞,一般,*,*,*,赤レンガ倉庫,アカレンガソウコ,アカレンガソーコ
名詞,固有名詞,一般,*,*,*,ランドマークタワー,ランドマークタワー,ランドマークタワー
名詞,固有名詞,一般,*,*,*,コスモワールド,コスモワールド,コスモワールド
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,固有名詞,一般,*,*,*,観光名所,カンコウメイショ,カンコーメイショ
名詞,一般,*,*,*,*,エリア,エリア,エリア
名詞,サ変接続,*,*,*,*,観光,カンコウ,カンコー
名詞,サ変接続,*,*,*,*,食事,ショクジ,ショクジ
名詞,サ変接続,*,*,*,*,買い物,カイモノ,カイモノ
名詞,数,*,*,*,*,一,イチ,イチ
名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
名詞,一般,*,*,*,*,子連れ,コヅレ,コズレ
名詞,固有名詞,一般,*,*,*,アンパンマンミュージアム,アンパンマンミュージアム,アンパンマンミュージアム
名詞,固有名詞,組織,*,*,*,三菱,ミツビシ,ミツビシ
名詞,固有名詞,地域,一般,*,*,みなとみらい,ミナトミライ,ミナトミライ
名詞,一般,*,*,*,*,技術,ギジュツ,ギジュツ
名詞,接尾,一般,*,*,*,館,カン,カン
名詞,固有名詞,一般,*,*,*,お勧め,オススメ,オススメ
名詞,一般,*,*,*,*,海,ウミ,ウミ
名詞,一般,*,*,*,*,目,メ,メ
名詞,副詞可能,*,*,*,*,前,マ

名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
名詞,一般,*,*,*,*,久しぶり,ヒサシブリ,ヒサシブリ
名詞,固有名詞,一般,*,*,*,コスモワールド,コスモワールド,コスモワールド
名詞,一般,*,*,*,*,赤レンガ,アカレンガ,アカレンガ
名詞,一般,*,*,*,*,一日中,イチニチジュウ,イチニチジュー
名詞,非自立,副詞可能,*,*,*,ところ,トコロ,トコロ
名詞,サ変接続,*,*,*,*,プレゼント,プレゼント,プレゼント
名詞,サ変接続,*,*,*,*,デート,デート,デート
名詞,固有名詞,一般,*,*,*,中華街,チュウカガイ,チューカガイ
名詞,副詞可能,*,*,*,*,夜,ヨル,ヨル
名詞,固有名詞,一般,*,*,*,中華街,チュウカガイ,チューカガイ
名詞,一般,*,*,*,*,ご飯,ゴハン,ゴハン
名詞,非自立,一般,*,*,*,の,ノ,ノ
名詞,一般,*,*,*,*,在処,アリカ,アリカ
名詞,一般,*,*,*,*,夜景,ヤケイ,ヤケイ
名詞,一般,*,*,*,*,広場,ヒロバ,ヒロバ
名詞,副詞可能,*,*,*,*,たくさん,タクサン,タクサン
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,固有名詞,地域,国,*,*,日本,ニッポン,ニッポン
名詞,サ変接続,*,*,*,*,代表,ダイヒョウ,ダイヒョー
名詞,接尾,形容動詞語幹,*,*,*,的,テキ,テキ
名詞,一般,*,*,*,*,港,ミナト,ミナト
名詞,一般,*,*,*,*,理由,リユウ,リユー
名詞,一般,*,*,*,*,友達,トモダチ,トモダチ
名詞,サ変接続,*,*,*,*,一緒,イッショ,イッショ
名詞,固有名詞,一般,*,*,*,観覧車,カンランシャ,カンランシャ
名詞,固有名詞,地域,一般,*,*,横浜,ヨコハマ,ヨコハマ
名詞,代名詞,一般,*,*,*,ここ,ココ,ココ
名詞,一般,*,*,*,*,定番,テイバン,テイバン
名詞,一般,*,*,*,*,夜景,ヤケイ,ヤケイ
名詞,形容動詞語幹,*,*,*,*,綺麗,キレイ,キレイ
名詞,一般,*,*,*,*,浜,ハマ,ハマ
名詞,固有名詞,一般,*,*,*,ですしおすし,デスシオスシ,デスシオスシ
名詞,形容動詞語幹,*,*,*,*,有名,ユウメイ,ユーメイ
名詞,一般,*,*,

KeyboardInterrupt: 

In [76]:
df

,spot_id,reviews,review_tokens
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...,"[最近, あたり, 仕事, プライベート, 用事, エリア, ビル, だらけ, 駅, 間, ..."
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...,"[ホタル, 観賞, 30分, 待ち, ストレス, ホタル, 観賞, 区域, カメラ, 発光体..."
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...,"[桟橋, 何, 無機質, 空間, 横浜港, 大さん橋, ここ, 国際, 客船, ターミナル,..."
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...,"[鎌倉, 大仏, 周囲, 緑, 山奥, よう, 雰囲気, 過去, 何度, 鎌倉大仏, 久しぶ..."
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...,"[創建, 住仁, 天皇, 三, 世紀, 古社, 相模国, 式内社, ケーブルカー, 駅, 数..."
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...,"[エレベーター, 展望台, 展望台, 一, 回, りし, 屋外, デッキ, 屋外, デッキ,..."
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...,"[航空路, 関係, 300m, の, そうだ, 高さ, 296m, あべのハルカス, 安定感..."
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...,"[他, イベント, 空き, 時間, つもり, 中, 迷路, みたい, マリンタワー, 氷川丸..."
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...,"[ここ, ゴー, 硫黄泉, 独特, ７, 年, 寿命, 黒玉子, 絶景, 富士山, お客様,..."
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...,"[帆船, 日, 本丸, みなと, 博物館, エリア, メモリアルパーク, よう, 日本丸, ..."


In [154]:
def get_spot_name(doc_id):
    spot_id = df.iloc[doc_id].spot_id
    spot = Spot.objects.get(id=spot_id)
    return spot.title

parallelize 参考 http://blog.adeel.io/2016/11/06/parallelize-pandas-map-or-apply/

In [77]:
# import numpy as np
# from multiprocessing import cpu_count, Pool
 
# cores = cpu_count() #Number of CPU cores on your system
# partitions = cores #Define as many partitions as you want
 
# def parallelize(data, func):
#     data_split = np.array_split(data, partitions)
#     pool = Pool(cores)
#     data = pd.concat(pool.map(func, data_split))
#     pool.close()
#     pool.join()
#     return data

In [78]:
# %time
# df = parallelize(df, lambda doc: reviews2tokensbysimpleparse(doc, stopwords))

In [79]:
# from dask import dataframe as dd
# from dask.multiprocessing import get
# from multiprocessing import cpu_count
# nCores = cpu_count()
# dd.from_pandas(df,npartitions=nCores).\
#    map_partitions(
#       lambda df : df.apply(
#          lambda doc: reviews2tokensbysimpleparse(doc, stopwords)), meta={'reviews': 'list'}).\
#    compute(get=get)

In [80]:
df

,spot_id,reviews,review_tokens
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...,"[最近, あたり, 仕事, プライベート, 用事, エリア, ビル, だらけ, 駅, 間, ..."
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...,"[ホタル, 観賞, 30分, 待ち, ストレス, ホタル, 観賞, 区域, カメラ, 発光体..."
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...,"[桟橋, 何, 無機質, 空間, 横浜港, 大さん橋, ここ, 国際, 客船, ターミナル,..."
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...,"[鎌倉, 大仏, 周囲, 緑, 山奥, よう, 雰囲気, 過去, 何度, 鎌倉大仏, 久しぶ..."
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...,"[創建, 住仁, 天皇, 三, 世紀, 古社, 相模国, 式内社, ケーブルカー, 駅, 数..."
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...,"[エレベーター, 展望台, 展望台, 一, 回, りし, 屋外, デッキ, 屋外, デッキ,..."
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...,"[航空路, 関係, 300m, の, そうだ, 高さ, 296m, あべのハルカス, 安定感..."
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...,"[他, イベント, 空き, 時間, つもり, 中, 迷路, みたい, マリンタワー, 氷川丸..."
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...,"[ここ, ゴー, 硫黄泉, 独特, ７, 年, 寿命, 黒玉子, 絶景, 富士山, お客様,..."
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...,"[帆船, 日, 本丸, みなと, 博物館, エリア, メモリアルパーク, よう, 日本丸, ..."


↓の参考：http://tdual.hatenablog.com/entry/2018/04/09/133000#gensim


In [83]:
docs = []
for index, row in df.iterrows():
    docs.append(row.review_tokens)

In [84]:
from gensim import corpora, models
d = corpora.Dictionary(docs)

In [90]:
# 使われている文書の数がno_belowより少ない単語を無視し、no_aboveの割合以上の文書に出てくる単語を無視している
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()

In [95]:
corpus = [d.doc2bow(w) for w in docs]
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus = corpus[test_size:]

In [97]:
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=50, passes=10)

using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 9026 documents, updating model once every 2000 documents, evaluating perplexity every 9026 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #18 (0.020): 0.005*"鎌倉" + 0.004*"お寺" + 0.004*"景色" + 0.003*"階段" + 0.003*"富士山" + 0.003*"観光" + 0.003*"平日" + 0.003*"海" + 0.003*"写真" + 0.002*"境内"
topic #28 (0.020): 0.005*"横浜" + 0.005*"海" + 0.004*"鎌倉" + 0.004*"景色" + 0.004*"東京タワー" + 0.004*"夜景" + 0.004*"レストラン" + 0.004*"夜" + 0.003*"展望台" + 0.003*"写真"
topic #23 (0.020): 0.008*"展示" + 0.005*"横浜" + 0.005*"美術館" + 0.004*"海" + 0.003*"景色" + 0.003*"レストラン" + 0.003*"平日" + 0.003*"写真" + 0.003*"イベント" + 0.003*"見学"
topic #47 (0.020): 0.007*"展示" + 0.005*"美術館" + 0.004*"レストラン" + 0.003*"横浜" + 0.003*"景色" + 0.003*"プー

topic #19 (0.020): 0.041*"展示" + 0.020*"美術館" + 0.011*"作品" + 0.009*"展" + 0.009*"館" + 0.008*"博物館" + 0.007*"見学" + 0.006*"室" + 0.006*"品" + 0.005*"館内"
topic #37 (0.020): 0.036*"神社" + 0.017*"紫陽花" + 0.016*"縁結び" + 0.015*"参拝" + 0.013*"階段" + 0.013*"パワースポット" + 0.012*"あじさい" + 0.011*"女性" + 0.011*"お参り" + 0.009*"熱海"
topic diff=2.089445, rho=0.391841
-8.680 per-word bound, 410.1 perplexity estimate based on a held-out corpus of 1026 documents with 213588 words
PROGRESS: pass 1, at document #9026/9026
merging changes from 1026 documents into a model of 9026 documents
topic #27 (0.020): 0.059*"古墳" + 0.041*"プラネタリウム" + 0.012*"自転車" + 0.011*"新青梅街道" + 0.009*"福生" + 0.009*"ジオラマ" + 0.008*"羽村市" + 0.008*"星空" + 0.007*"階段" + 0.007*"鉄道模型"
topic #7 (0.020): 0.010*"席" + 0.009*"観戦" + 0.009*"イベント" + 0.007*"東京駅" + 0.005*"ドーム" + 0.005*"試合" + 0.005*"開催" + 0.005*"球場" + 0.004*"レストラン" + 0.004*"野球"
topic #24 (0.020): 0.035*"アトラクション" + 0.024*"遊園地" + 0.021*"乗り物" + 0.014*"大人" + 0.012*"イルミネーション" + 0.010*"入園" + 0.008*"園内" + 0.008*"家

topic #39 (0.020): 0.053*"海" + 0.016*"船" + 0.015*"お台場" + 0.012*"景色" + 0.010*"レインボーブリッジ" + 0.008*"海岸" + 0.008*"夜" + 0.007*"東京湾" + 0.006*"夏" + 0.006*"きれい"
topic #4 (0.020): 0.012*"街" + 0.011*"商店街" + 0.009*"屋" + 0.006*"お土産" + 0.005*"最近" + 0.005*"購入" + 0.004*"観光客" + 0.004*"買い物" + 0.004*"品" + 0.004*"活気"
topic #1 (0.020): 0.134*"中華街" + 0.049*"横浜" + 0.040*"ラーメン" + 0.035*"中華" + 0.026*"元町" + 0.025*"中国" + 0.024*"横浜中華街" + 0.023*"食べ放題" + 0.018*"肉まん" + 0.018*"中華料理"
topic diff=0.480612, rho=0.342735
PROGRESS: pass 3, at document #8000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #5 (0.020): 0.031*"温泉" + 0.028*"風呂" + 0.013*"湯" + 0.012*"銭湯" + 0.011*"サウナ" + 0.010*"露天風呂" + 0.008*"お湯" + 0.008*"料金" + 0.007*"休憩" + 0.007*"岩盤浴"
topic #45 (0.020): 0.033*"コース" + 0.026*"山" + 0.020*"ハイキング" + 0.019*"山頂" + 0.019*"富士山" + 0.013*"頂上" + 0.012*"階段" + 0.011*"登山" + 0.011*"自然" + 0.010*"高尾山"
topic #16 (0.020): 0.023*"レストラン" + 0.019*"階" + 0.017*"買い物" + 0.015*"地下" + 0.013*"ショップ" + 0.010*"店舗" 

topic #25 (0.020): 0.256*"鎌倉" + 0.067*"鎌倉駅" + 0.066*"江ノ電" + 0.049*"鶴岡八幡宮" + 0.028*"長谷寺" + 0.027*"小町通り" + 0.025*"八幡宮" + 0.018*"藤沢" + 0.015*"長谷駅" + 0.014*"北鎌倉"
topic #3 (0.020): 0.037*"渋谷" + 0.033*"待ち合わせ" + 0.029*"像" + 0.026*"城" + 0.019*"ハチ公" + 0.018*"渋谷駅" + 0.017*"小田原城" + 0.016*"天守閣" + 0.013*"歌舞伎町" + 0.013*"小田原"
topic #43 (0.020): 0.069*"プール" + 0.038*"球場" + 0.031*"大仏" + 0.030*"野球" + 0.019*"試合" + 0.014*"スタジアム" + 0.013*"観戦" + 0.012*"大仏様" + 0.011*"夏" + 0.011*"野球場"
topic #32 (0.020): 0.092*"庭園" + 0.020*"池" + 0.018*"都会" + 0.017*"築地" + 0.016*"日本庭園" + 0.012*"都心" + 0.012*"真ん中" + 0.011*"新橋駅" + 0.011*"築地市場" + 0.010*"入園"
topic diff=0.358840, rho=0.308416
PROGRESS: pass 5, at document #6000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #24 (0.020): 0.046*"アトラクション" + 0.019*"大人" + 0.019*"乗り物" + 0.016*"遊園地" + 0.010*"パレード" + 0.009*"家族" + 0.008*"待ち時間" + 0.008*"平日" + 0.008*"夜" + 0.008*"入園"
topic #32 (0.020): 0.094*"庭園" + 0.022*"池" + 0.018*"築地" + 0.017*"日本庭園" + 0.017*"都会" +

topic #6 (0.020): 0.085*"横浜" + 0.038*"みなとみらい" + 0.027*"山下公園" + 0.018*"観光" + 0.017*"赤レンガ倉庫" + 0.016*"夜景" + 0.015*"船" + 0.015*"横浜駅" + 0.011*"港" + 0.010*"ベイブリッジ"
topic #7 (0.020): 0.019*"イベント" + 0.013*"席" + 0.010*"開催" + 0.007*"会場" + 0.007*"観戦" + 0.006*"東京駅" + 0.005*"トイレ" + 0.005*"ドーム" + 0.005*"アクセス" + 0.004*"屋根"
topic #38 (0.020): 0.065*"川崎駅" + 0.064*"川崎" + 0.046*"川崎大師" + 0.040*"秋葉原" + 0.032*"大師" + 0.028*"飴" + 0.028*"アニメ" + 0.026*"護摩" + 0.020*"風鈴" + 0.017*"だるま"
topic #27 (0.020): 0.067*"プラネタリウム" + 0.034*"古墳" + 0.029*"科学" + 0.019*"宇宙" + 0.018*"模型" + 0.018*"ジオラマ" + 0.017*"科学館" + 0.016*"実験" + 0.013*"鉄道模型" + 0.013*"体験"
topic #33 (0.020): 0.058*"ホール" + 0.031*"席" + 0.028*"コンサート" + 0.021*"劇場" + 0.015*"階" + 0.014*"舞台" + 0.012*"演奏" + 0.012*"音楽" + 0.011*"歌舞伎" + 0.011*"公演"
topic diff=0.428472, rho=0.282696
PROGRESS: pass 7, at document #4000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #7 (0.020): 0.016*"イベント" + 0.014*"東京駅" + 0.013*"席" + 0.009*"開催" + 0.007*"観戦" + 0.0

PROGRESS: pass 8, at document #9026/9026
merging changes from 1026 documents into a model of 9026 documents
topic #28 (0.020): 0.034*"スカイツリー" + 0.032*"展望台" + 0.025*"景色" + 0.022*"展望" + 0.022*"東京タワー" + 0.018*"夜景" + 0.018*"東京" + 0.016*"エレベーター" + 0.014*"富士山" + 0.012*"眺め"
topic #37 (0.020): 0.121*"神社" + 0.032*"参拝" + 0.026*"境内" + 0.026*"鳥居" + 0.025*"お参り" + 0.019*"参道" + 0.017*"初詣" + 0.015*"本殿" + 0.012*"階段" + 0.011*"御朱印"
topic #39 (0.020): 0.054*"海" + 0.019*"船" + 0.018*"お台場" + 0.015*"景色" + 0.010*"レインボーブリッジ" + 0.009*"東京湾" + 0.008*"海岸" + 0.007*"風" + 0.007*"夏" + 0.007*"きれい"
topic #33 (0.020): 0.052*"ホール" + 0.034*"席" + 0.024*"コンサート" + 0.023*"劇場" + 0.016*"階" + 0.016*"舞台" + 0.015*"歌舞伎" + 0.011*"音楽" + 0.011*"公演" + 0.010*"音響"
topic #42 (0.020): 0.039*"and" + 0.032*"the" + 0.032*"to" + 0.027*"a" + 0.020*"of" + 0.020*"was" + 0.014*"I" + 0.014*"in" + 0.013*"we" + 0.012*"tour"
topic diff=0.206676, rho=0.272035
PROGRESS: pass 9, at document #2000/9026
merging changes from 2000 documents into a model of 902

In [98]:
lda

In [99]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

perplexity = np.exp2(-lda.log_perplexity(test_corpus))
print("perplexity:", perplexity)


N:  1049471


-8.197 per-word bound, 293.4 perplexity estimate based on a held-out corpus of 902 documents with 1049471 words


perplexity: 293.3792761947933


In [100]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

for t in range(10):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")


Topic #  0
家: 0.09421459585428238
甲州街道: 0.09265514463186264
洋館: 0.058474063873291016
本陣: 0.043151628226041794
山手: 0.037175364792346954
根津神社: 0.029010571539402008
平塚: 0.02035052888095379
名主: 0.01663219928741455
イタリア: 0.015676848590373993
井伊直弼: 0.014763345010578632


Topic #  1
中華街: 0.10048764199018478
横浜: 0.03218351677060127
中華: 0.02979511395096779
ラーメン: 0.0296112559735775
中国: 0.0242722500115633
元町: 0.0192877110093832
横浜中華街: 0.018493657931685448
食べ放題: 0.018011396750807762
食事: 0.014644848182797432
中華料理: 0.014490026980638504


Topic #  2
箱根: 0.09489552676677704
芦ノ湖: 0.056717973202466965
富士山: 0.04092371091246605
湖: 0.025314325466752052
海賊船: 0.0219790730625391
近藤勇: 0.02020767703652382
箱根神社: 0.018121635541319847
新選組: 0.016957299783825874
関所: 0.014497892931103706
遊覧船: 0.014364239759743214


Topic #  3
像: 0.03795330971479416
渋谷: 0.0347520150244236
待ち合わせ: 0.03392067179083824
城: 0.021733369678258896
ハチ公: 0.017772361636161804
渋谷駅: 0.016982393339276314
銅像: 0.015684673562645912
大学: 0.01336301304399

In [101]:
topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
for t in topics[:10]:
    print("{}: {}".format(t[0], t[1]))

6: 0.37545087933540344
8: 0.1800699234008789
39: 0.09178987890481949
16: 0.06934396177530289
24: 0.05645284801721573
23: 0.04636848345398903
7: 0.04317484796047211
4: 0.02594909258186817
12: 0.019414125010371208
28: 0.01695297472178936


In [102]:
for t in topics[:10]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")

Topic #  6
横浜: 0.07909324020147324
みなとみらい: 0.03446449711918831
山下公園: 0.02412276156246662
観光: 0.01946979947388172
赤レンガ倉庫: 0.01566310040652752
夜景: 0.01505365315824747
船: 0.01492425799369812
横浜駅: 0.01352753583341837
港: 0.010845786891877651
景色: 0.009272082708775997


Topic #  8
ビル: 0.02625218592584133
レストラン: 0.020274734124541283
イルミネーション: 0.014380395412445068
夜: 0.012119932100176811
東京: 0.011103303171694279
カフェ: 0.010109459981322289
東京駅: 0.009986980818212032
素敵: 0.00905646476894617
食事: 0.009051525965332985
新宿: 0.008843234740197659


Topic #  39
海: 0.053727343678474426
船: 0.019869796931743622
お台場: 0.018721094354987144
景色: 0.015231834724545479
レインボーブリッジ: 0.009656188078224659
東京湾: 0.008602851070463657
海岸: 0.008332064375281334
風: 0.007495739031583071
夏: 0.007289670407772064
きれい: 0.006833498831838369


Topic #  16
レストラン: 0.02297155000269413
階: 0.0204001497477293
買い物: 0.018225444480776787
ショップ: 0.014275514520704746
地下: 0.01412828080356121
店舗: 0.01186271570622921
直結: 0.009815772995352745
ショッピング: 

固有名詞ない方がよさそう？
* 地域性を除去したい
* ただし、富士山とかはどうなの？
  * ランドマークを残すパターンと両方除去するパターンでやってみる

## 両方除去するパターン

In [105]:
def reviews2tokensbysimpleparsewithoutareanouns(reviews, stopwords):
    tokens = []
    for review in reviews:
        for chunk in mecab.parse(review).splitlines()[:-1]:
            (surface, feature) = chunk.split('\t')
            if feature.startswith('名詞'):
                if feature.startswith('名詞,固有名詞,地域'):
                    pass
                else:
#                     print(feature)
                    tokens.append(surface)
    return tokens

In [106]:
%%time
stopwords = ['\n','、','「','」']
stopwords += slothlib_stopwords
df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokensbysimpleparsewithoutareanouns(doc, stopwords))

CPU times: user 47.6 s, sys: 1.07 s, total: 48.7 s
Wall time: 48.7 s


In [110]:
df.review_tokens

0       [最近, あたり, 仕事, プライベート, 用事, エリア, ビル, だらけ, 駅, 間, ...
1       [ホタル, 観賞, 30分, 待ち, ストレス, ホタル, 観賞, 区域, カメラ, 発光体...
2       [桟橋, 何, 無機質, 空間, 横浜港, 大さん橋, ここ, 国際, 客船, ターミナル,...
3       [大仏, 周囲, 緑, 山奥, よう, 雰囲気, 過去, 何度, 鎌倉大仏, 久しぶり, 今...
4       [創建, 住仁, 天皇, 三, 世紀, 相模国, 式内社, ケーブルカー, 駅, 数, 分,...
5       [エレベーター, 一, 回, りし, 屋外, デッキ, 屋外, デッキ, 方, 風を感じて,...
6       [航空路, 関係, 300m, の, そうだ, 高さ, 296m, あべのハルカス, 安定感...
7       [他, イベント, 空き, 時間, つもり, 中, 迷路, みたい, マリンタワー, 氷川丸...
8       [ここ, ゴー, 硫黄泉, 独特, ７, 年, 寿命, 黒玉子, 絶景, 富士山, お客様,...
9       [帆船, 日, 本丸, 博物館, エリア, よう, 日本丸, 博物館, セット, 入場券, ...
10      [日産, 最新, 車, 販売店, 気軽, こと, 土日, イベント, 子供, よう, 工夫,...
11      [駅, 江ノ島電鉄, 無人駅, 駅, ホーム, 海, 開放, 感, 駅, 造り, レトロ, ...
12      [場所, それ, 長い, 通り, ありません, 異国情緒, 港町, 印象, よう, 場所, ...
13      [イベント, 時, 訪問, ため, どこ, 店, 混雑, 行列, 雰囲気, 時, 利用, 海...
14      [敷地内, の, 天守閣, 中, 城, 歴史, よう, 現代, 的, 美術, これ, 残念,...
15      [紫陽花, 後, 10時, 半, 頃, の, 多く, 人, 賑わい, 浴衣, レンタル, 人...
16      [表通り, 入り口, 北野, 様, 雰囲気, 異国情緒, 雰囲気, 様, 期待, ゴールデン...
17      [ハマ, っ

In [107]:
%time
docs = []
for index, row in df.iterrows():
    docs.append(row.review_tokens)
from gensim import corpora, models
d = corpora.Dictionary(docs)
# 使われている文書の数がno_belowより少ない単語を無視し、no_aboveの割合以上の文書に出てくる単語を無視している
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()
corpus = [d.doc2bow(w) for w in docs]
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus = corpus[test_size:]
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=50, passes=10)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 13.1 µs


adding document #0 to Dictionary(0 unique tokens: [])
built Dictionary(135233 unique tokens: ['0', '1', '100', '1000円', '100万']...) from 9026 documents (total 6531543 corpus positions)
discarding 99186 tokens: [('11分', 4), ('1891年', 4), ('2400', 2), ('280m', 1), ('296m', 4), ('29枚', 2), ('69階', 3), ('7900円', 1), ('APEC', 2), ('BOOKS', 2)]...
keeping 36047 tokens which were in no less than 5 and no more than 1805 (=20.0%) documents
resulting dictionary: Dictionary(36047 unique tokens: ['0', '1', '100', '1000円', '100万']...)
using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 9026 documents, updating model once every 2000 documents, evaluating perplexity every 9026 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #39 (0.020): 0

topic diff=2.429524, rho=0.391841
PROGRESS: pass 1, at document #8000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #42 (0.020): 0.008*"市場" + 0.008*"値段" + 0.007*"屋" + 0.007*"マグロ" + 0.007*"魚" + 0.007*"料理" + 0.006*"寿司" + 0.006*"野菜" + 0.006*"街" + 0.006*"味"
topic #15 (0.020): 0.027*"遊具" + 0.014*"アスレチック" + 0.014*"滑り台" + 0.013*"遺跡" + 0.011*"広場" + 0.010*"城" + 0.009*"子供たち" + 0.009*"古墳" + 0.008*"復元" + 0.008*"小学校"
topic #46 (0.020): 0.031*"and" + 0.028*"to" + 0.024*"the" + 0.024*"a" + 0.018*"was" + 0.018*"of" + 0.016*"I" + 0.015*"class" + 0.011*"cooking" + 0.009*"is"
topic #36 (0.020): 0.068*"見学" + 0.021*"洋館" + 0.017*"工場" + 0.016*"ビール" + 0.016*"ツアー" + 0.016*"説明" + 0.014*"予約" + 0.014*"参加" + 0.012*"工場見学" + 0.012*"内部"
topic #23 (0.020): 0.031*"ケーブルカー" + 0.025*"馬" + 0.016*"競馬" + 0.015*"帝釈天" + 0.015*"ロープウェイ" + 0.011*"レース" + 0.009*"リフト" + 0.007*"一里塚" + 0.006*"競馬場" + 0.006*"登山"
topic diff=2.117627, rho=0.391841
-8.558 per-word bound, 376.9 perplexity estimate based on a 

topic #34 (0.020): 0.058*"ロープウェイ" + 0.053*"富士山" + 0.041*"黒" + 0.030*"たまご" + 0.028*"大涌谷" + 0.023*"硫黄" + 0.023*"卵" + 0.017*"個" + 0.013*"桃源台" + 0.012*"駐車場"
topic diff=0.686953, rho=0.342735
PROGRESS: pass 3, at document #6000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #29 (0.020): 0.073*"中華街" + 0.014*"スタッフ" + 0.012*"マッサージ" + 0.010*"門" + 0.009*"食べ放題" + 0.009*"コース" + 0.008*"山下公園" + 0.008*"中華" + 0.008*"友達" + 0.008*"バー"
topic #19 (0.020): 0.033*"観戦" + 0.025*"野球" + 0.024*"球場" + 0.020*"試合" + 0.020*"席" + 0.013*"スタジアム" + 0.012*"東京ドーム" + 0.011*"応援" + 0.011*"ドーム" + 0.010*"イベント"
topic #49 (0.020): 0.061*"お寺" + 0.029*"本堂" + 0.025*"境内" + 0.014*"寺" + 0.014*"観音" + 0.011*"堂" + 0.010*"寺院" + 0.008*"山門" + 0.008*"拝観" + 0.008*"参道"
topic #14 (0.020): 0.076*"地蔵" + 0.049*"大山" + 0.037*"巣鴨" + 0.028*"とげぬき地蔵" + 0.024*"麒麟" + 0.021*"旧東海道" + 0.018*"街道" + 0.015*"下社" + 0.015*"女坂" + 0.015*"高岩寺"
topic #22 (0.020): 0.043*"案内" + 0.041*"観光" + 0.026*"アンテナショップ" + 0.017*"日本酒" + 0.014*"パンフレット" +

topic #11 (0.020): 0.094*"お金" + 0.045*"ダム" + 0.032*"銭" + 0.027*"銀行" + 0.024*"お札" + 0.024*"線香" + 0.018*"lesson" + 0.017*"築地" + 0.017*"トンネル" + 0.015*"ざる"
topic #9 (0.020): 0.049*"マリーナ" + 0.041*"蕨" + 0.040*"神奈川県庁" + 0.032*"税関" + 0.032*"塔" + 0.027*"ヨット" + 0.026*"海老名" + 0.025*"横浜税関" + 0.024*"横穴" + 0.024*"本庁舎"
topic diff=0.558468, rho=0.308416
PROGRESS: pass 5, at document #4000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #20 (0.020): 0.061*"海" + 0.013*"船" + 0.012*"コース" + 0.012*"イベント" + 0.010*"ベンチ" + 0.009*"海岸" + 0.009*"夏" + 0.008*"夜" + 0.008*"港" + 0.006*"整備"
topic #27 (0.020): 0.053*"お台場" + 0.052*"教会" + 0.037*"フジテレビ" + 0.028*"ガンダム" + 0.022*"ゆりかもめ" + 0.019*"氷川丸" + 0.016*"聖堂" + 0.015*"自由の女神" + 0.013*"湯島聖堂" + 0.012*"聖橋"
topic #47 (0.020): 0.059*"赤レンガ倉庫" + 0.041*"ラーメン" + 0.032*"赤レンガ" + 0.032*"倉庫" + 0.027*"イベント" + 0.025*"桜木町" + 0.024*"ランドマークタワー" + 0.021*"地区" + 0.017*"かまぼこ" + 0.013*"みなとみらい線"
topic #35 (0.020): 0.035*"飛行機" + 0.030*"コース" + 0.027*"山" + 0.022*"ハイキング"

topic #37 (0.020): 0.025*"大人" + 0.019*"乗り物" + 0.018*"プール" + 0.017*"イルミネーション" + 0.014*"アトラクション" + 0.012*"遊園地" + 0.012*"園内" + 0.011*"円" + 0.011*"入園" + 0.011*"入場料"
topic #47 (0.020): 0.050*"赤レンガ倉庫" + 0.034*"ラーメン" + 0.031*"赤レンガ" + 0.031*"倉庫" + 0.027*"イベント" + 0.024*"地区" + 0.023*"ランドマークタワー" + 0.021*"桜木町" + 0.014*"かまぼこ" + 0.012*"みなとみらい線"
topic #43 (0.020): 0.069*"神社" + 0.024*"境内" + 0.018*"参拝" + 0.016*"お参り" + 0.014*"鳥居" + 0.010*"参道" + 0.009*"初詣" + 0.009*"本殿" + 0.007*"階段" + 0.007*"御朱印"
topic diff=0.222876, rho=0.294717
PROGRESS: pass 7, at document #2000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #33 (0.020): 0.033*"景色" + 0.024*"夜景" + 0.019*"東京タワー" + 0.016*"スカイツリー" + 0.014*"夜" + 0.013*"展望" + 0.013*"眺め" + 0.011*"富士山" + 0.010*"天気" + 0.009*"船"
topic #11 (0.020): 0.093*"お金" + 0.043*"ダム" + 0.036*"銭" + 0.026*"銀行" + 0.024*"お札" + 0.023*"築地" + 0.023*"線香" + 0.019*"トンネル" + 0.019*"lesson" + 0.015*"水"
topic #0 (0.020): 0.029*"池" + 0.020*"緑" + 0.018*"自然" + 0.009*"園内" + 0.00

topic #29 (0.020): 0.069*"中華街" + 0.014*"中華" + 0.014*"料理" + 0.012*"スタッフ" + 0.011*"食事" + 0.010*"コース" + 0.010*"食べ放題" + 0.010*"マッサージ" + 0.008*"友達" + 0.008*"門"
topic #43 (0.020): 0.067*"神社" + 0.024*"境内" + 0.020*"参拝" + 0.017*"お参り" + 0.015*"鳥居" + 0.010*"参道" + 0.009*"初詣" + 0.009*"本殿" + 0.008*"階段" + 0.007*"御朱印"
topic #24 (0.020): 0.146*"城" + 0.079*"天守閣" + 0.060*"Mac" + 0.043*"小田原城" + 0.027*"甲冑" + 0.022*"北条氏" + 0.020*"鎧" + 0.018*"戦国時代" + 0.016*"忍者" + 0.016*"刀"
topic diff=0.198977, rho=0.272035
-8.187 per-word bound, 291.5 perplexity estimate based on a held-out corpus of 1026 documents with 205727 words
PROGRESS: pass 8, at document #9026/9026
merging changes from 1026 documents into a model of 9026 documents
topic #22 (0.020): 0.048*"観光" + 0.048*"案内" + 0.022*"アンテナショップ" + 0.020*"日本酒" + 0.015*"パンフレット" + 0.014*"お酒" + 0.011*"販売" + 0.010*"情報" + 0.010*"酒" + 0.009*"野菜"
topic #14 (0.020): 0.068*"地蔵" + 0.030*"巣鴨" + 0.030*"大山" + 0.028*"中山道" + 0.026*"旧東海道" + 0.026*"街道" + 0.025*"本陣" + 0.022*"京急" + 0.020*"塩

In [108]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

perplexity = np.exp2(-lda.log_perplexity(test_corpus))
print("perplexity:", perplexity)

def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

for t in range(10):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")
    
topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
for t in topics[:10]:
    print("{}: {}".format(t[0], t[1]))

for t in topics[:10]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")


N:  1000413


-8.231 per-word bound, 300.5 perplexity estimate based on a held-out corpus of 902 documents with 1000413 words


perplexity: 300.5295715040566
Topic #  0
池: 0.02895812690258026
緑: 0.02122555859386921
自然: 0.016667645424604416
芝生: 0.008957202546298504
園内: 0.008310711942613125
庭園: 0.007765164133161306
季節: 0.007490850985050201
木々: 0.006954898592084646
整備: 0.006936879362910986
都会: 0.006751617416739464


Topic #  1
ビル: 0.03884385898709297
地下: 0.01971721649169922
階: 0.019146045669913292
レストラン: 0.017530417069792747
待ち合わせ: 0.013280270621180534
イルミネーション: 0.010119752958416939
カフェ: 0.007285031955689192
街: 0.006462349556386471
飲食店: 0.006128499284386635
イベント: 0.006100022699683905


Topic #  2
温泉: 0.03817528113722801
風呂: 0.03426626697182655
湯: 0.01600859872996807
露天風呂: 0.013217581436038017
サウナ: 0.012884307652711868
お湯: 0.009821712970733643
銭湯: 0.009750205092132092
料金: 0.008410366252064705
岩盤浴: 0.007492103613913059
休憩: 0.0072546061128377914


Topic #  3
買い物: 0.02050025761127472
レストラン: 0.018754450604319572
ショップ: 0.013559392653405666
店舗: 0.012382393702864647
ショッピング: 0.010703750886023045
食事: 0.00934516079723835
フード

### 固有名詞を全て除く

In [121]:
def reviews2tokensbysimpleparsewithoutpropernouns(reviews, stopwords):
    tokens = []
    for review in reviews:
        for chunk in mecab.parse(review).splitlines()[:-1]:
            (surface, feature) = chunk.split('\t')
            if feature.startswith('名詞'):
                if feature.startswith('名詞,固有名詞'):
                    pass
                else:
#                     print(feature)
                    tokens.append(surface)
    return tokens
stopwords = ['\n','、','「','」']
stopwords += slothlib_stopwords
df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokensbysimpleparsewithoutpropernouns(doc, stopwords))

In [123]:
docs = []
for index, row in df.iterrows():
    docs.append(row.review_tokens)
from gensim import corpora, models
d = corpora.Dictionary(docs)
# 使われている文書の数がno_belowより少ない単語を無視し、no_aboveの割合以上の文書に出てくる単語を無視している
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()
corpus = [d.doc2bow(w) for w in docs]
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus = corpus[test_size:]
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=50, passes=10)

adding document #0 to Dictionary(0 unique tokens: [])
built Dictionary(71402 unique tokens: ['0', '1', '100', '11', '12']...) from 9026 documents (total 5187333 corpus positions)
discarding 49253 tokens: [('2400', 2), ('BOOKS', 1), ('BayBike', 1), ('Birkenstock', 1), ('Daughnuts', 1), ('Holic', 2), ('Jog', 1), ('LANDMARK', 1), ('Location', 1), ('MARINE', 1)]...
keeping 22149 tokens which were in no less than 5 and no more than 1805 (=20.0%) documents
resulting dictionary: Dictionary(22149 unique tokens: ['0', '1', '100', '11', '12']...)
using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 9026 documents, updating model once every 2000 documents, evaluating perplexity every 9026 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/9026
merging changes from 2000 documents into a model of 9026 documents
topic

topic #10 (0.020): 0.118*"飛行機" + 0.024*"ターミナル" + 0.024*"格納庫" + 0.024*"空港" + 0.017*"機体" + 0.014*"機" + 0.011*"間近" + 0.011*"整備" + 0.009*"着陸" + 0.009*"航空機"
topic #8 (0.020): 0.014*"園内" + 0.013*"芝生" + 0.013*"遊具" + 0.013*"花" + 0.012*"広場" + 0.011*"桜" + 0.010*"季節" + 0.009*"時期" + 0.008*"弁当" + 0.007*"バラ"
topic #41 (0.020): 0.099*"城" + 0.037*"城跡" + 0.032*"灯台" + 0.023*"標柱" + 0.014*"天守閣" + 0.013*"階段" + 0.013*"城址" + 0.013*"常磐" + 0.013*"跡" + 0.012*"山"
topic #45 (0.020): 0.050*"水族館" + 0.028*"水槽" + 0.023*"ペンギン" + 0.021*"門" + 0.016*"魚" + 0.014*"イルカ" + 0.012*"ショー" + 0.009*"皇居" + 0.009*"アシカ" + 0.009*"クラゲ"
topic diff=2.040833, rho=0.391841
-8.100 per-word bound, 274.3 perplexity estimate based on a held-out corpus of 1026 documents with 170099 words
PROGRESS: pass 1, at document #9026/9026
merging changes from 1026 documents into a model of 9026 documents
topic #35 (0.020): 0.034*"池" + 0.032*"紅葉" + 0.014*"緑" + 0.010*"自然" + 0.010*"時期" + 0.010*"季節" + 0.009*"桜" + 0.009*"周り" + 0.009*"ボート" + 0.008*"都会"
topic #4

topic #47 (0.020): 0.019*"アトラクション" + 0.014*"大人" + 0.012*"ショー" + 0.010*"平日" + 0.010*"入場" + 0.009*"混雑" + 0.008*"予約" + 0.007*"乗り物" + 0.006*"チケット" + 0.006*"ランド"
topic #27 (0.020): 0.081*"美術館" + 0.051*"展示" + 0.047*"作品" + 0.027*"展" + 0.018*"鑑賞" + 0.013*"カフェ" + 0.012*"絵" + 0.011*"絵画" + 0.011*"美術" + 0.008*"彫刻"
topic #6 (0.020): 0.029*"橋" + 0.015*"像" + 0.013*"写真" + 0.013*"待ち合わせ" + 0.011*"雷門" + 0.010*"観光" + 0.009*"仲見世" + 0.008*"スポット" + 0.007*"交差点" + 0.007*"下町"
topic #14 (0.020): 0.026*"船" + 0.023*"外観" + 0.020*"教会" + 0.015*"内部" + 0.014*"建築" + 0.011*"レンガ" + 0.011*"赤レンガ" + 0.010*"見学" + 0.010*"重厚" + 0.008*"煉瓦"
topic diff=0.499643, rho=0.342735
PROGRESS: pass 3, at document #8000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #47 (0.020): 0.019*"アトラクション" + 0.015*"大人" + 0.012*"ショー" + 0.010*"入場" + 0.010*"平日" + 0.009*"混雑" + 0.008*"予約" + 0.008*"乗り物" + 0.007*"チケット" + 0.006*"ランド"
topic #16 (0.020): 0.031*"庭" + 0.027*"庭園" + 0.021*"洋館" + 0.020*"家" + 0.015*"館" + 0.013*"建築" + 0.0

topic #19 (0.020): 0.028*"展示" + 0.019*"品" + 0.012*"展" + 0.011*"庭園" + 0.010*"国宝" + 0.010*"庭" + 0.009*"本館" + 0.009*"今回" + 0.008*"入場" + 0.008*"お茶"
topic #37 (0.020): 0.033*"レストラン" + 0.026*"ビル" + 0.021*"階" + 0.018*"地下" + 0.015*"ショップ" + 0.014*"カフェ" + 0.011*"食事" + 0.011*"ショッピング" + 0.011*"イルミネーション" + 0.010*"買い物"
topic #10 (0.020): 0.134*"飛行機" + 0.041*"ターミナル" + 0.038*"空港" + 0.016*"ラウンジ" + 0.015*"出発" + 0.013*"機" + 0.013*"国際線" + 0.013*"離発着" + 0.013*"デッキ" + 0.012*"離着陸"
topic #0 (0.020): 0.048*"見学" + 0.042*"展示" + 0.018*"館" + 0.014*"説明" + 0.013*"記念" + 0.011*"写真" + 0.010*"資料" + 0.009*"資料館" + 0.008*"ガイド" + 0.008*"室"
topic diff=0.355468, rho=0.308416
PROGRESS: pass 5, at document #6000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #48 (0.020): 0.157*"梅" + 0.022*"時期" + 0.020*"まつり" + 0.017*"梅林" + 0.016*"木" + 0.014*"満開" + 0.011*"開花" + 0.010*"季節" + 0.010*"香り" + 0.010*"園内"
topic #10 (0.020): 0.134*"飛行機" + 0.039*"ターミナル" + 0.037*"空港" + 0.017*"ラウンジ" + 0.014*"出発" + 0.014*"機" + 0

topic #20 (0.020): 0.047*"席" + 0.031*"ホール" + 0.021*"イベント" + 0.018*"会場" + 0.018*"コンサート" + 0.012*"劇場" + 0.011*"座席" + 0.010*"トイレ" + 0.009*"開催" + 0.009*"ライブ"
topic #44 (0.020): 0.017*"地" + 0.016*"跡" + 0.013*"説明" + 0.013*"碑" + 0.012*"板" + 0.009*"石碑" + 0.009*"当時" + 0.008*"名前" + 0.006*"看板" + 0.005*"道路"
topic #41 (0.020): 0.123*"城" + 0.085*"天守閣" + 0.050*"灯台" + 0.033*"丘" + 0.031*"高台" + 0.031*"跡" + 0.028*"堀" + 0.027*"石垣" + 0.025*"城跡" + 0.020*"階段"
topic #23 (0.020): 0.065*"観光" + 0.061*"案内" + 0.019*"アンテナショップ" + 0.018*"パンフレット" + 0.016*"南口" + 0.015*"品" + 0.014*"階" + 0.013*"販売" + 0.012*"情報" + 0.012*"地図"
topic diff=0.379408, rho=0.282696
PROGRESS: pass 7, at document #4000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #27 (0.020): 0.085*"美術館" + 0.055*"展示" + 0.047*"作品" + 0.027*"展" + 0.018*"鑑賞" + 0.016*"カフェ" + 0.011*"絵" + 0.011*"絵画" + 0.011*"美術" + 0.009*"館内"
topic #39 (0.020): 0.090*"本" + 0.081*"図書館" + 0.034*"資料館" + 0.024*"資料" + 0.021*"郷土" + 0.017*"おもちゃ" + 0.017*"室" + 0.0

topic #21 (0.020): 0.133*"コース" + 0.060*"皇居" + 0.048*"自転車" + 0.024*"練習" + 0.018*"ランニング" + 0.018*"ランナー" + 0.013*"ジョギング" + 0.012*"交通" + 0.010*"キロ" + 0.009*"サイクリング"
topic #2 (0.020): 0.029*"夜" + 0.017*"街" + 0.010*"イベント" + 0.010*"ホテル" + 0.009*"最近" + 0.009*"食事" + 0.009*"猫" + 0.009*"昼間" + 0.008*"気分" + 0.007*"平日"
topic #10 (0.020): 0.135*"飛行機" + 0.031*"ターミナル" + 0.028*"空港" + 0.024*"航空" + 0.015*"機" + 0.015*"ラウンジ" + 0.014*"格納庫" + 0.013*"機体" + 0.012*"着陸" + 0.012*"整備"
topic #44 (0.020): 0.017*"地" + 0.015*"跡" + 0.014*"説明" + 0.013*"碑" + 0.013*"板" + 0.009*"石碑" + 0.008*"名前" + 0.008*"当時" + 0.006*"看板" + 0.006*"帝釈天"
topic diff=0.191986, rho=0.272035
PROGRESS: pass 9, at document #2000/9026
merging changes from 2000 documents into a model of 9026 documents
topic #9 (0.020): 0.080*"海" + 0.044*"プール" + 0.024*"海岸" + 0.020*"夏" + 0.019*"花火" + 0.017*"イルミネーション" + 0.011*"砂浜" + 0.010*"波" + 0.009*"有料" + 0.009*"冬"
topic #30 (0.020): 0.056*"ビール" + 0.031*"街" + 0.022*"試飲" + 0.020*"工場" + 0.013*"屋" + 0.011*"路地" + 0.011*"日本

In [124]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

perplexity = np.exp2(-lda.log_perplexity(test_corpus))
print("perplexity:", perplexity)

def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

# 10個ごとの
for t in range(10):
    print("Topic # ",t)s
    get_topic_words(t)
    print("\n")
    
topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
for t in topics[:10]:
    print("{}: {}".format(t[0], t[1]))

for t in topics[:10]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")

N:  788389


-7.911 per-word bound, 240.7 perplexity estimate based on a held-out corpus of 902 documents with 788389 words


perplexity: 240.70555392270765
Topic #  0
展示: 0.04985745996236801
見学: 0.04976479709148407
館: 0.023898808285593987
記念: 0.01730266772210598
説明: 0.01405995711684227
写真: 0.012036735191941261
資料: 0.011535114608705044
資料館: 0.011201690882444382
再現: 0.00786199327558279
室: 0.007841356098651886


Topic #  1
動物: 0.047747764736413956
パンダ: 0.021271388977766037
川: 0.02098717913031578
園内: 0.01394769549369812
水: 0.012851018458604813
自然: 0.011041710153222084
たち: 0.010090712457895279
ライオン: 0.008514844812452793
園: 0.008513528853654861
遊歩道: 0.00810482632368803


Topic #  2
夜: 0.0282210074365139
街: 0.01729791983962059
イベント: 0.009974236600100994
ホテル: 0.00982996728271246
最近: 0.009651238098740578
食事: 0.00954046007245779
昼間: 0.008558513596653938
猫: 0.008254003711044788
気分: 0.007981949485838413
平日: 0.0070185051299631596


Topic #  3
屋: 0.014487693086266518
購入: 0.009379412978887558
値段: 0.008548539131879807
野菜: 0.008007154799997807
味: 0.007882382720708847
買い物: 0.006675227545201778
食事: 0.0064851040951907635
料理: 0.

In [156]:
# all_topics = lda.get_document_topics(corpus, minimum_probability=0)

類似度算出

In [159]:
from gensim import similarities
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

scanning corpus to determine the number of features (consider setting `num_features` explicitly)
creating matrix with 9026 documents and 50 features


In [184]:
c = corpus[0]
vec_lda = lda[c]
vec_lda
s  = doc_index .__getitem__(vec_lda)
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)
s[:10]

[(0, 1.0000001),
 (752, 0.96468407),
 (2588, 0.9038274),
 (5133, 0.8825784),
 (13, 0.878334),
 (2664, 0.87463665),
 (2698, 0.87235385),
 (5760, 0.853515),
 (3170, 0.8462695),
 (3697, 0.84394944)]

In [205]:
def get_recommends(base_doc_id):
    c = corpus[base_doc_id]
    vec_lda = lda[c]
    s  = doc_index .__getitem__(vec_lda)
    s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)
    spot_ids = []
    for doc_id, sim in s[1:10]:
        spot_ids.append(df.iloc[doc_id].spot_id)    
    return Spot.objects.filter(id__in=spot_ids)

<QuerySet [<Spot: spot=理科ハウス (逗子市), count=2/2>, <Spot: spot=Honda ウェルカムプラザ青山 (港区), count=43/43>, <Spot: spot=JPタワー学術文化総合ミュージアムインターメディアテク (千代田区), count=55/55>, <Spot: spot=アド・ミュージアム東京 (港区), count=18/18>, <Spot: spot=パイオニア プラザ銀座 (中央区), count=10/6>, <Spot: spot=ペンステーションミュージアム (中央区), count=18/18>, <Spot: spot=郵政博物館 (墨田区), count=40/30>, <Spot: spot=一丁目プレイランド (港区), count=1/1>, <Spot: spot=東京アニメセンター (千代田区), count=8/8>]>


In [207]:
def show_recommends(base_doc_id):
    print('base spot name: {}'.format(get_spot_name(base_doc_id)))
    recommends = get_recommends(base_doc_id)
    for spot in recommends:
        print(spot.title)

In [213]:
show_recommends(2133)

base spot name: S.WAVE (大磯町)
バリ風天然温泉 スパ リブール ヨコハマ (横浜市)
スカイスパ YOKOHAMA (横浜市)
おふろの王様 高座渋谷駅前店 (大和市)
湘南台温泉 らく (藤沢市)
府中駅前天然温泉 縄文の湯 (府中市)
メンズサウナ こり・こり (新宿区)
天然温泉 楽天地スパ (墨田区)
タイ古式マッサージ・オイルマッサージ スッカパープディー (立川市)
河辺温泉 梅の湯 (青梅市)


In [194]:
len(similarity_matrix[10])

9026

## 考えなければいけないこと
* なぜこのスポットが抽出されたかを実際の類似度が高かったトピックも表示してみたりする
* neologdってどうなの？
  * 横浜、横浜駅は「名詞,固有名詞,地域,一般」、桜木町は「名詞,固有名詞,一般」
    * 「駅」、「町」が付くものは弾くようにする
* 範囲を名詞から広げる
* word2vecとかどうなのか